# Demo example

Demo example using Lu Zhang's spatial data with 1,020,000 locations. The Julia code is at <https://github.com/Hua-Zhou/NGPP.jl>. It can be installed at package manager:
```
] add https://github.com/Hua-Zhou/NGPP.jl
```

In [1]:
versioninfo()

Julia Version 1.2.0
Commit c6da87ff4b (2019-08-20 00:03 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i7-6920HQ CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)
Environment:
  JULIA_EDITOR = code


In [2]:
using FileIO, IterativeSolvers, JLD2, LinearAlgebra, NGPP, RData, SparseArrays, UnicodePlots

## Retrieve data

Data is provided by Lu Zhang in the R data file `data_cleaned_small_expanded.RData` and a JLD file `MAPtest.jld`.

In [3]:
;ls

MAPtest.jld
Manifest.toml
Project.toml
data_cleaned_small_expanded.RData
demo_onemillion.ipynb
runtests.jl


In [4]:
data_cleaned = load("data_cleaned_small_expanded.RData", convert = true)["data_cleaned_small"]

,x,y,scaled_x,scaled_y,NDVI,EVI,red_reflectance
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-1.05614e7,4.44757e6,0.557829,1.11149,0.638163,0.414689,0.0165
2,-1.05517e7,4.44757e6,0.567558,1.11149,0.602785,0.397433,0.0271
3,-1.05508e7,4.44757e6,0.568485,1.11149,0.61713,0.412772,0.0233
4,-1.05503e7,4.44757e6,0.568948,1.11149,0.619339,0.423305,0.0236
5,-1.05485e7,4.44757e6,0.570801,1.11149,0.608896,0.421535,0.0272
6,-1.05475e7,4.44757e6,0.571728,1.11149,0.600099,0.426182,0.0315
7,-1.05471e7,4.44757e6,0.572191,1.11149,0.593327,0.431977,0.0356
8,-1.05466e7,4.44757e6,0.572655,1.11149,0.602894,0.409723,0.0283
9,-1.05452e7,4.44757e6,0.574044,1.11149,0.545865,0.372666,0.0461


In [5]:
n = size(data_cleaned, 1)
m, r, p̃ = 5, 9, 2
y = data_cleaned[:, :GPP]
X = [ones(n) data_cleaned[:, :LE]]
coord = convert(Matrix, data_cleaned[:, 3:4])
jlddata = load("MAPtest.jld")
nn = jlddata["NN"]
Dgpknot = jlddata["D_gpp_knots"]
Dgptall = jlddata["D_gpp_tall"];

## NNGP data structure

First let form an instance of type `NNGP` from data:

In [6]:
nngp = NNGP(X, X, y, coord, r, m, nn.nnIndx, nn.nnIndxLU, Dgpknot, Dgptall)

NNGP{Float64}(5, 1020000, 2, 2, 9, [1.0 6.2738199580540375; 1.0 6.348264483234865; … ; 1.0 3.8918202981106265; 1.0 3.8918202981106265], [1.0 6.2738199580540375; 1.0 6.348264483234865; … ; 1.0 3.8918202981106265; 1.0 3.8918202981106265], [-3.1292636661785136, -3.0480787539262, -3.0261914810386994, -2.9749497343714624, -2.9947327732209077, -2.9897502018764435, -2.9897502018764435, -3.025161084244803, -3.171276846068922, -2.9808436610602405  …  -4.940642922276221, -4.947660494934867, -5.013138424314374, -4.947660494934867, -4.940642922276221, -5.013138424314374, -4.933674252960127, -4.866534950122499, -4.933674252960127, -4.976233867378923], [0.557828510699613 1.1114872069504718; 0.5675580777466986 1.1114872069504718; … ; 1.1082440179347788 0.0; 1.11009726880089 0.0], Array{Int64,1}[[], [1], [2, 1], [3, 2, 1], [4, 3, 2, 1], [5, 4, 3, 2, 1], [6, 5, 4, 3, 2], [7, 6, 5, 4, 3], [8, 7, 6, 5, 4], [9, 8, 7, 6, 5]  …  [1019392, 1019391, 1019990, 1018776, 1018777], [1019393, 1018777, 1019394, 1019

This structure contains all data needed for computation and model parameters. It take around 1GB memory.

In [7]:
Base.summarysize(nngp)

1036323784

In [8]:
fieldnames(typeof(nngp))

(:m, :n, :p, :p̃, :r, :X, :X̃, :y, :coord, :nnlist, :nndist, :Dgpknot, :Dgptall, :β, :δ²gp, :δ²nngp, :ϕgp, :ϕnngp, :A, :D⁻½, :Znr, :Zrr, :Cnn, :Cknot, :Ctall)

Let's set parameters to the values in Lu Zhang's demo and update the arrays `A`, `D`, and `Z`, which depend on parameter values. Evalution of `A`, `D` and `Z` involves evaluating a large number of Matern kernels, which can be potentially sped up by MKL's Vector Math Library (VML).

In [9]:
fill!(nngp.ϕgp, 5)
fill!(nngp.ϕnngp, 5)
fill!(nngp.δ²gp, 5)
fill!(nngp.δ²nngp, 5)
@time update_AD!(nngp)
@time update_Z!(nngp)

  7.906943 seconds (129.17 M allocations: 1.950 GiB, 7.75% gc time)
  3.391786 seconds (55.85 M allocations: 878.885 MiB, 2.24% gc time)


## Xstar matrix

Evaluation of the posterior involves heavy linear algebra of the Xstar matrix. `NNGPXstar` is a wrapper type for this matrix and is a subtype of `AbstractMatrix`. It uses _little_ extra memory beyond what's in NNGP type. 

In [10]:
xstar = NNGPXstar(nngp)
Base.summarysize(xstar)

1044484024

In [11]:
size(xstar)

(3060018, 2040020)

We can perform linear algebra directly on the wrapper type `NNGPXstar`. For example, use the LSMR iterative solver to get least squares solution:

In [12]:
ystar = [  y; zeros(p̃ * (n + r))]
sol = [X \ y; zeros(p̃ * (n + r))]
@time lsmr!(sol, xstar, ystar, verbose=false)

 23.242282 seconds (2.73 M allocations: 243.405 MiB, 1.35% gc time)


2040020-element Array{Float64,1}:
  -8.09319681967134    
   0.7108539227066691  
   0.4618423932976787  
   0.2721972519629294  
   1.72669254106056    
  -1.672843587587533   
  -2.2410309584288393  
  -1.6905989796359753  
  -1.285108620133151   
   0.6548392037655622  
 -11.575382275119091   
  -0.07215557995414612 
  -0.021680301250155257
   ⋮                   
   0.0888577438492604  
   0.08112319976319522 
   0.07896365844411726 
   0.06824663036367436 
   0.06606591959596644 
   0.06329404337432216 
   0.06713327144217276 
   0.07927023534204546 
   0.0897979565936053  
   0.07342348553616859 
   0.07087753737905765 
   0.06374577553577519 

It takes 186 iterations. Both the runtime and solution match those provided by Lu Zhang. The advantage here is we did not create extra memory for a sparse matrix for linear algebra. Everything is done using original data directly.

In [13]:
# objective value
norm(xstar * sol - ystar)

74.339310133922

## Sparse Xstar

If needed, we can create a sparse matrix of Xstar in CSC format. 

In [14]:
xstar_sp = sparse(xstar)

3060018×2040020 SparseMatrixCSC{Float64,Int64} with 34680060 stored entries:
  [1      ,       1]  =  1.0
  [2      ,       1]  =  1.0
  [3      ,       1]  =  1.0
  [4      ,       1]  =  1.0
  [5      ,       1]  =  1.0
  [6      ,       1]  =  1.0
  [7      ,       1]  =  1.0
  [8      ,       1]  =  1.0
  [9      ,       1]  =  1.0
  [10     ,       1]  =  1.0
  [11     ,       1]  =  1.0
  [12     ,       1]  =  1.0
  ⋮
  [3060013, 2040015]  =  7.66156
  [3060014, 2040015]  =  -1.06569
  [1019996, 2040016]  =  3.6763
  [3060014, 2040016]  =  6.32816
  [1019997, 2040017]  =  3.72569
  [3060015, 2040017]  =  6.92947
  [1019998, 2040018]  =  3.9982
  [3060016, 2040018]  =  6.98379
  [3060017, 2040018]  =  -1.07446
  [1019999, 2040019]  =  3.89182
  [3060017, 2040019]  =  5.68966
  [1020000, 2040020]  =  3.89182
  [3060018, 2040020]  =  5.78769

In [15]:
Base.summarysize(xstar_sp)

571201288

In [16]:
UnicodePlots.spy(xstar_sp)

                 Sparsity Pattern
           ┌────────────────────────────┐    
         1 │⡟⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
           │⡇⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
           │⡇⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀│    
           │⡇⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀│    
           │⡇⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀│    
           │⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀│    
           │⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦│    
           │⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⠀⠀⠀⠀⠀⠀⠀⠉⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀│    
           │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀│    
           │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣦⡀⠀⠀⠀│    
           │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⠀⠀│    


Running iterative solver on sparse matrix should give the same answer as using xstar.

In [17]:
ystar = [  y; zeros(p̃ * (n + r))]
sol = [X \ y; zeros(p̃ * (n + r))]
@time lsmr!(sol, xstar_sp, ystar, verbose=false)

 27.256769 seconds (1.09 M allocations: 162.468 MiB)


2040020-element Array{Float64,1}:
  -8.066071555080358   
   0.705180665876815   
   0.441546599517484   
   0.23548223062400303 
   1.6774996394461839  
  -1.6925042756923165  
  -2.2405965999348183  
  -1.7335713128525179  
  -1.3035084587889452  
   0.6323679101238959  
 -11.534457024012      
  -0.06800857437799794 
  -0.013703421437443714
   ⋮                   
   0.08809435887426728 
   0.08020155122288283 
   0.07793048460148685 
   0.06727034879076074 
   0.06509218628864623 
   0.06239919891762024 
   0.066281828445705   
   0.07828762104260827 
   0.08876968629148897 
   0.0725943926833649  
   0.07006559774503879 
   0.06287020190926053 

In [18]:
# objective value
norm(xstar_sp * sol - ystar)

74.35234737331376

## Gram matrix

Alternatively we can form the Gram matrix `Xstar'Xstar`:

In [19]:
G_sp = xstar_sp'xstar_sp

2040020×2040020 SparseMatrixCSC{Float64,Int64} with 113919108 stored entries:
  [1      ,       1]  =  1.02e6
  [2      ,       1]  =  4.86172e6
  [3      ,       1]  =  46099.6
  [4      ,       1]  =  68849.4
  [5      ,       1]  =  1.19727e5
  [6      ,       1]  =  41785.8
  [7      ,       1]  =  37130.3
  [8      ,       1]  =  90429.1
  [9      ,       1]  =  35572.7
  [10     ,       1]  =  57443.0
  [11     ,       1]  =  1.03776e5
  [12     ,       1]  =  2.2527e5
  ⋮
  [15     , 2040020]  =  -0.00995971
  [16     , 2040020]  =  -0.0734813
  [17     , 2040020]  =  0.229461
  [18     , 2040020]  =  -0.000939958
  [19     , 2040020]  =  -0.0161677
  [20     , 2040020]  =  0.0510119
  [1020020, 2040020]  =  3.89182
  [2038137, 2040020]  =  -8.07638
  [2038138, 2040020]  =  2.35675
  [2038139, 2040020]  =  1.96394
  [2038807, 2040020]  =  -0.709328
  [2039423, 2040020]  =  -28.9561
  [2040020, 2040020]  =  48.6436

In [20]:
Base.summarysize(G_sp)

1839026056

In [21]:
UnicodePlots.spy(G_sp)

                        Sparsity Pattern
           ┌──────────────────────────────────────────┐    
         1 │⡿⣯⡉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠻⣍⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉│ > 0
           │⡇⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
           │⡇⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⡇⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⡇⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⡇⠀⠀⠀⠀⠀⠀⠀⠀⠈⠿⣧⡤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠋⠿⣧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀│    
           │⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀│    
           │⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀│    
           │⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀│    
           │⣧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳│    
           │⡇⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⡇⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⡇⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
           │⡇⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    

Conjugate gradient takes an exessive number of iterations on this example.

In [24]:
# ycg = xstar_sp'ystar
# sol = [X \ y; zeros(p̃ * (n + r))]
# @time cg!(sol, G, ycg, verbose=true)

In [25]:
# # objective value
# norm(xstar_sp * sol - ystar)

## Sparse Cholesky on the Gram matrix

CHOLMOD library only works on `SparseMatrixCSC{Float64,Int64}`. Other float or integer type does not work.

In [22]:
@time Gchol = cholesky(G_sp)

 32.794658 seconds (402.91 k allocations: 11.929 GiB, 2.19% gc time)


SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  supernodal
maxnnz:  0
nnz:     398726550
success: true


In [23]:
xtystar = xstar_sp'ystar
sol = [X \ y; zeros(p̃ * (n + r))]
@time sol_spchol = Gchol \ xtystar

  2.470425 seconds (181.27 k allocations: 71.413 MiB, 0.31% gc time)


2040020-element Array{Float64,1}:
 -8.012056471452093   
  0.6957832340168568  
 -0.6701658153648793  
 -0.004727283897980831
  0.5140839100635148  
 -1.5734148343168914  
 -3.2357707969362335  
 -2.2263062467176713  
 -1.4674843597092635  
 -1.377229535266219   
 -7.67338524575885    
  0.06073634073990105 
 -0.03448551801241786 
  ⋮                   
 -0.007440088983492607
 -0.0173330318048248  
 -0.020850138628352954
 -0.030022604928570593
 -0.03153700022207326 
 -0.03234420553098315 
 -0.026607055611314812
 -0.014816784751767412
  0.001385137068239263
 -0.004917691569870858
 -0.005375398273233044
 -0.009924621733335637

In [24]:
# objective value
# better solution than LSMR
norm(xstar * sol_spchol - ystar)

71.56033593415685

In [28]:
Gchol2 = deepcopy(Gchol)
@time cholesky!(Gchol2, G_sp)

 29.706675 seconds (9.91 k allocations: 4.366 GiB, 0.07% gc time)


SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  supernodal
maxnnz:  0
nnz:     398726550
success: true


In [29]:
xtystar = xstar_sp'ystar
sol = [X \ y; zeros(p̃ * (n + r))]
@time sol_spchol = Gchol2 \ xtystar

  2.603747 seconds (16 allocations: 62.279 MiB, 0.15% gc time)


2040020-element Array{Float64,1}:
 -8.012056471452093   
  0.6957832340168568  
 -0.6701658153648793  
 -0.004727283897980831
  0.5140839100635148  
 -1.5734148343168914  
 -3.2357707969362335  
 -2.2263062467176713  
 -1.4674843597092635  
 -1.377229535266219   
 -7.67338524575885    
  0.06073634073990105 
 -0.03448551801241786 
  ⋮                   
 -0.007440088983492607
 -0.0173330318048248  
 -0.020850138628352954
 -0.030022604928570593
 -0.03153700022207326 
 -0.03234420553098315 
 -0.026607055611314812
 -0.014816784751767412
  0.001385137068239263
 -0.004917691569870858
 -0.005375398273233044
 -0.009924621733335637

## Sparse LDLt on Gram matrix

In [30]:
@time Gldlt = ldlt(G_sp)

373.353567 seconds (51.94 k allocations: 26.216 GiB, 1.18% gc time)


SuiteSparse.CHOLMOD.Factor{Float64}
type:    LDLt
method:  simplicial
maxnnz:  1345124474
nnz:     375398277
success: true


## GPU computing on Xstar

Let's first transfer data to GPU. GPU prefers single precision.

In [26]:
using CuArrays
using CuArrays.CUSPARSE
using CuArrays.CUSOLVER

In [27]:
d_ystar = CuArray(Float32.([  y; zeros(p̃ * (n + r))]))
d_xstar = CuSparseMatrixCSR(SparseMatrixCSC{Float32, Int32}(xstar_sp))
d_sol   = CuArray(Float32.([X \ y; zeros(p̃ * (n + r))]));

In [28]:
# # unfortunately the lsmr! function is very slow on GPU (could be fixed)
# @time lsmr!(d_sol, d_xstar, d_ystar, verbose=true, maxiter=186)

In [29]:
# not sure why it throws a DimensionMismatch error
CUSOLVER.csrlsqvqr!(SparseMatrixCSC{Float32, Int32}(xstar_sp), Float32.(ystar), sol, Float32(1e-4), 'O')

MethodError: MethodError: no method matching csrlsqvqr!(::SparseMatrixCSC{Float32,Int32}, ::Array{Float32,1}, ::Array{Float64,1}, ::Float32, ::Char)
Closest candidates are:
  csrlsqvqr!(::SparseMatrixCSC{Float32,Ti} where Ti<:Integer, ::Array{Float32,1}, !Matched::Array{Float32,1}, ::Float32, ::Char) at /home/huazhou/.julia/packages/CuArrays/wXQp8/src/solver/sparse.jl:154
  csrlsqvqr!(!Matched::SparseMatrixCSC{Float64,Ti} where Ti<:Integer, !Matched::Array{Float64,1}, ::Array{Float64,1}, !Matched::Float64, ::Char) at /home/huazhou/.julia/packages/CuArrays/wXQp8/src/solver/sparse.jl:154
  csrlsqvqr!(!Matched::SparseMatrixCSC{Complex{Float32},Ti} where Ti<:Integer, !Matched::Array{Complex{Float32},1}, !Matched::Array{Complex{Float32},1}, ::Float32, ::Char) at /home/huazhou/.julia/packages/CuArrays/wXQp8/src/solver/sparse.jl:154
  ...

## GPU computing on Gram matrix G

Let's try GPU computing on the sparse Gram matrix:

In [31]:
d_G = CuSparseMatrixCSR(SparseMatrixCSC{Float32, Int32}(G_sp))
d_ycg = CuArray(Float32.(xstar_sp'ystar))
d_sol   = CuArray(Float32.([X \ y; zeros(p̃ * (n + r))]))
# CG terminates prematurely for single precision (same as on CPU)
cg!(d_sol, d_G, d_ycg, verbose=true)

  1	1.36e+05
  2	8.69e+04
  3	3.58e+04
  4	1.63e+04
  5	2.54e+04
  6	1.23e+04
  7	4.59e+03



2040020-element CuArray{Float32,1}:
 -8.049442    
  0.7295005   
 -0.010146505 
  0.0047746804
  0.06041367  
  0.0008856331
 -0.069686666 
 -0.009277315 
  0.0040515577
  0.004048425 
 -0.058975674 
 -0.016982332 
  0.029808676 
  ⋮           
  1.5609656e-5
  8.915857e-6 
  1.2207955e-5
  8.238352e-6 
  9.5038695e-6
  7.5606526e-6
  7.5779058e-6
  1.2314799e-5
  1.4571909e-5
  8.752527e-6 
  9.136707e-6 
  7.046345e-6 

In double precision, it takes excessively number of iterations (same as on CPU).

In [32]:
# d_G = CuSparseMatrixCSR(G_sp)
# d_ycg = CuArray(ycg)
# d_sol   = CuArray([X \ y; zeros(p̃ * (n + r))]);
# cg!(d_sol, d_G, d_ycg, verbose=true)

Perform sparse Cholesky on GPU

In [33]:
# d_G = CuSparseMatrixCSR(SparseMatrixCSC{Float32, Int32}(G_sp))
# d_ycg = CuArray(Float32.(ycg))
# d_sol   = CuArray(Float32.([X \ y; zeros(p̃ * (n + r))]))
# CUSOLVER.csrlsvchol!(d_G, d_ycg, d_sol, Float32(1e-4), zero(Cint), 'O')

Perform sparse QR on GPU

In [34]:
# d_G = CuSparseMatrixCSR(SparseMatrixCSC{Float32, Int32}(G_sp))
# d_ycg = CuArray(Float32.(ycg))
# d_sol   = CuArray(Float32.([X \ y; zeros(p̃ * (n + r))]))
# CUSOLVER.csrlsvqr!(d_G, d_ycg, d_sol, Float32(1e-4), one(Cint), 'O')